In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install mediapipe


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import matplotlib.pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles

NameError: name 'audio_classifier' is not defined

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('models/gun_detector.h5')


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands

# Open the webcam
cap = cv2.VideoCapture(0)

# Ensure saved images have unique names
output_dir = "captured_images"
os.makedirs(output_dir, exist_ok=True)

# Initialize MediaPipe Hands with a minimum detection and tracking confidence
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Convert the image to RGB for processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        results = hands.process(image_rgb)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Check if hand landmarks are detected
        if results.multi_hand_landmarks and results.multi_handedness:
            hand_images = {}  # Store cropped images by hand label
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine hand label (Left/Right)
                hand_label = handedness.classification[0].label  # "Left" or "Right"

                # Get coordinates of middle finger tip (Landmark index 12) and palm joint (Landmark index 0)
                middle_finger_tip = hand_landmarks.landmark[12]
                palm_joint = hand_landmarks.landmark[0]

                h, w, _ = image.shape
                middle_finger_x = int(middle_finger_tip.x * w)
                middle_finger_y = int(middle_finger_tip.y * h)
                palm_x = int(palm_joint.x * w)
                palm_y = int(palm_joint.y * h)

                # Calculate the distance between the middle finger tip and palm joint
                distance = ((middle_finger_x - palm_x) ** 2 + (middle_finger_y - palm_y) ** 2) ** 0.5

                # Define the bounding box around the hand with dynamic padding based on distance
                padding = int(distance * 0.7)  # Adjust padding dynamically
                x_min = max(int(min([lm.x for lm in hand_landmarks.landmark]) * w) - padding, 0)
                y_min = max(int(min([lm.y for lm in hand_landmarks.landmark]) * h) - padding, 0)
                x_max = min(int(max([lm.x for lm in hand_landmarks.landmark]) * w) + padding, w)
                y_max = min(int(max([lm.y for lm in hand_landmarks.landmark]) * h) + padding, h)

                # Draw the bounding box
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

                # Crop the hand image
                cropped_hand = image[y_min:y_max, x_min:x_max]
                hand_images[hand_label] = cropped_hand

                # Resize the image to 224x224
                resized_image = cv2.resize(cropped_hand, (256, 256))

                # Convert the image to RGB (3 channels)
                resized_image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

                # Predicting the class of the image
                prediction = model.predict(np.expand_dims(resized_image_rgb, axis=0))

                print(f"Prediction: {prediction[0]}")

                if prediction[0] >= 0.5:
                    cv2.putText(image, "Gun not Detected", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                elif prediction[0] < 0.5:
                    cv2.putText(image, "Gun Detected", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Print middle finger coordinates
                print(f"{hand_label} Hand - Middle Finger Tip Coordinates: x = {middle_finger_x}, y = {middle_finger_y}")

        # Display the frame with bounding boxes
        cv2.imshow('MediaPipe Hands', image)

        # Save hand images and exit loop when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            if 'Left' in hand_images:
                left_file_name = os.path.join(output_dir, "left_hand.jpg")
                cv2.imwrite(left_file_name, hand_images['Left'])
                print(f"Saved {left_file_name}")
            if 'Right' in hand_images:
                right_file_name = os.path.join(output_dir, "right_hand.jpg")
                cv2.imwrite(right_file_name, hand_images['Right'])
                print(f"Saved {right_file_name}")
            break
        

# Release the capture and close the windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 268, y = 47
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [1.]
Right Hand - Middle Finger Tip Coordinates: x = 254, y = 68
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 266, y = 125
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 269, y = 137
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 265, y = 150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 264, y = 155
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 250, y = 126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 250, y = 142
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction: [1.]
Left Hand - Middle Finger

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands

# Open the webcam
cap = cv2.VideoCapture('6037257_people_person_man_h771dcd52A002C0271128LNV10108720p5000br.mp4')

# Ensure saved images have unique names
output_dir = "captured_images"
os.makedirs(output_dir, exist_ok=True)

# Initialize MediaPipe Hands with a minimum detection and tracking confidence
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Convert the image to RGB for processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        results = hands.process(image_rgb)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Check if hand landmarks are detected
        if results.multi_hand_landmarks and results.multi_handedness:
            hand_images = {}  # Store cropped images by hand label
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                # Determine hand label (Left/Right)
                hand_label = handedness.classification[0].label  # "Left" or "Right"

                # Get coordinates of middle finger tip (Landmark index 12) and palm joint (Landmark index 0)
                middle_finger_tip = hand_landmarks.landmark[12]
                palm_joint = hand_landmarks.landmark[0]

                h, w, _ = image.shape
                middle_finger_x = int(middle_finger_tip.x * w)
                middle_finger_y = int(middle_finger_tip.y * h)
                palm_x = int(palm_joint.x * w)
                palm_y = int(palm_joint.y * h)

                # Calculate the distance between the middle finger tip and palm joint
                distance = ((middle_finger_x - palm_x) ** 2 + (middle_finger_y - palm_y) ** 2) ** 0.5

                # Define the bounding box around the hand with dynamic padding based on distance
                padding = int(distance * 1)  # Adjust padding dynamically
                x_min = max(int(min([lm.x for lm in hand_landmarks.landmark]) * w) - padding, 0)
                y_min = max(int(min([lm.y for lm in hand_landmarks.landmark]) * h) - padding, 0)
                x_max = min(int(max([lm.x for lm in hand_landmarks.landmark]) * w) + padding, w)
                y_max = min(int(max([lm.y for lm in hand_landmarks.landmark]) * h) + padding, h)

                # Draw the bounding box
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

                # Crop the hand image
                cropped_hand = image[y_min:y_max, x_min:x_max]
                hand_images[hand_label] = cropped_hand

                # Resize the image to 224x224
                resized_image = cv2.resize(cropped_hand, (256, 256))

                # Convert the image to RGB (3 channels)
                resized_image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

                # Predicting the class of the image
                prediction = model.predict(np.expand_dims(resized_image_rgb, axis=0))
                # prediction = np.argmax(prediction, axis=1)

                print(f"Prediction: {prediction[0]}")

                if prediction[0] >= 0.5:
                    cv2.putText(image, "Gun not Detected", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                elif prediction[0] < 0.5:
                    cv2.putText(image, "Gun Detected", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    
                # Print middle finger coordinates
                print(f"{hand_label} Hand - Middle Finger Tip Coordinates: x = {middle_finger_x}, y = {middle_finger_y}")

        # Display the frame with bounding boxes
        cv2.imshow('MediaPipe Hands', image)

        # Save hand images and exit loop when video ends
        if cv2.waitKey(10) & 0xFF == ord('q'):
            if 'Left' in hand_images:
                left_file_name = os.path.join(output_dir, "left_hand.jpg")
                cv2.imwrite(left_file_name, hand_images['Left'])
                print(f"Saved {left_file_name}")
            if 'Right' in hand_images:
                right_file_name = os.path.join(output_dir, "right_hand.jpg")
                cv2.imwrite(right_file_name, hand_images['Right'])
                print(f"Saved {right_file_name}")
            break

# Release the capture and close the windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 299, y = 519
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction: [0.]
Left Hand - Middle Finger Tip Coordinates: x = 266, y = 475
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction: [0.]
Left Hand - Middle Finger Tip Coordinates: x = 253, y = 451
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction: [0.]
Left Hand - Middle Finger Tip Coordinates: x = 261, y = 435
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [1.]
Right Hand - Middle Finger Tip Coordinates: x = 325, y = 397
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: [0.]
Left Hand - Middle Finger Tip Coordinates: x = 293, y = 401
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction: [0.]
Right Hand - Middle Finger Tip Coordinates: x = 292, y = 357
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction: [1.]
Left Hand - Middle Finger Tip Coordinates: x = 329, y = 381
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: [0.]
Right Hand - Middle Fin